In [42]:
import pulp
import sys
import preprocess_data
import importlib
import GA
import pandas as pd

In [44]:
importlib.reload(preprocess_data)
input_file = "./csv/conference_schedule_2021.csv"
parsed_input_file, papers_schedule = preprocess_data.parse_csv_to_array(input_file)
session_durations, num_tracks_per_session = preprocess_data.session_details(parsed_input_file)
session_details = [(session_durations[i], num_tracks_per_session[i]) for i in range(len(session_durations))]
print(session_details)

[(50.0, 2), (50.0, 1), (50.0, 2), (50.0, 2), (50.0, 1), (50.0, 2), (50.0, 1), (50.0, 2), (50.0, 1), (50.0, 2), (50.0, 1), (50.0, 2), (50.0, 1)]


In [45]:
papers_dict = preprocess_data.create_papers(input_file)
papers = papers_array = [x for x in papers_dict.values()]

for i in range(5):
    print(papers[i].print_paper())

ID: 1, Authors: ['Egor Spirin', ' Egor Bogomolov', ' Vladimir Kovalenko', ' Timofey Bryksin', '  Pre-print'], Duration: 3.0, Topic: 1
ID: 2, Authors: ['Maria Papoutsoglou', ' Johannes Wachs', ' Georgia Kapitsaki', '  Pre-print'], Duration: 3.0, Topic: 1
ID: 3, Authors: ['Nikolai Sviridov', ' Mikhail Evtikhiev', ' Vladimir Kovalenko', '  Pre-print'], Duration: 3.0, Topic: 1
ID: 4, Authors: ['Ahmed Zerouali', ' Camilo Velázquez-Rodríguez', ' Coen De Roover', '  Pre-print', '  Media Attached'], Duration: 3.0, Topic: 1
ID: 5, Authors: ['Ozren Dabic', ' Emad Aghajani', ' Gabriele Bavota', '  Pre-print'], Duration: 3.0, Topic: 1


In [46]:
num_sessions = len(session_details)
max_tracks = max(num_of_tracks for _, num_of_tracks in session_details)
num_papers = len(papers)

print(num_papers, num_sessions, max_tracks)

112 13 2


In [47]:
# Set up the problem
prob = pulp.LpProblem("Conference_Schedule", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("paper_schedule", 
                          [(i, j, k) for i in range(num_papers)
                                     for j in range(num_sessions)
                                     for k in range(max_tracks)],
                          cat='Binary')

# Objective function - As before, we want to minimize the total number of slots used
prob += pulp.lpSum(x[i, j, k] for i in range(num_papers) for j in range(num_sessions) for k in range(max_tracks))

# Constraints

# Constraint 1: Total duration of papers in a session cannot exceed session duration
for j, (session_duration, num_of_tracks) in enumerate(session_details):
    for k in range(num_of_tracks):
        prob += pulp.lpSum(x[i, j, k] * papers[i].duration for i in range(num_papers)) <= session_duration


In [48]:
def have_common_authors(paper1, paper2):
    return not set(paper1.authors).isdisjoint(paper2.authors)


In [49]:
for i in range(num_papers):
    for j in range(i+1, num_papers):
        if have_common_authors(papers[i], papers[j]):
            for session in range(num_sessions):
                for track_i in range(session_details[session][1]):
                    for track_j in range(session_details[session][1]):
                        if track_i != track_j:
                            prob += x[i, session, track_i] + x[j, session, track_j] <= 1


# Constraint 3: Papers with the same topic should be scheduled in the same session
for topic in set(paper.topic for paper in papers):
    for j in range(num_sessions):
        # For each topic, find papers with that topic and ensure they are scheduled in the same session
        topic_papers = [i for i, paper in enumerate(papers) if paper.topic == topic]
        for i in topic_papers:
            for k in range(session_details[j][1]):
                prob += pulp.lpSum(x[i, j, m] for m in range(session_details[j][1])) == x[i, j, k]

In [50]:
# Solve the problem
if prob.solve() == pulp.LpStatusInfeasible:
    # Problem is infeasible; relax some constraints and try again
    print("Infeasible problem. Relaxing constraints for diagnostic purpose.")

status = prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/5k/15bqvj117x74tp3qhvsy1m880000gn/T/bd961e6a7e0c417ea97e658d3173e02f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/5k/15bqvj117x74tp3qhvsy1m880000gn/T/bd961e6a7e0c417ea97e658d3173e02f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 46169 COLUMNS
At line 148762 RHS
At line 194927 BOUNDS
At line 197840 ENDATA
Problem MODEL has 46164 rows, 2912 columns and 91616 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.02 seconds
Cgl0002I 1568 variables fixed
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0 to -1.79769e+308
Probing was t

In [54]:
# Output results or debug information
if status == pulp.LpStatusOptimal:
    for v in prob.variables():
        print(pulp.LpStatusOptimal)
        if v.varValue == 0:
            print(v.name, "=", v.varValue)
elif status == pulp.LpStatusInfeasible:
    print("Problem is still infeasible after relaxation.")
else:
    print("Problem could not be solved. Status:", pulp.LpStatus[status])

1
paper_schedule_(0,_0,_0) = 0.0
1
paper_schedule_(0,_0,_1) = 0.0
1
paper_schedule_(0,_1,_0) = 0.0
1
paper_schedule_(0,_1,_1) = 0.0
1
paper_schedule_(0,_10,_0) = 0.0
1
paper_schedule_(0,_10,_1) = 0.0
1
paper_schedule_(0,_11,_0) = 0.0
1
paper_schedule_(0,_11,_1) = 0.0
1
paper_schedule_(0,_12,_0) = 0.0
1
paper_schedule_(0,_12,_1) = 0.0
1
paper_schedule_(0,_2,_0) = 0.0
1
paper_schedule_(0,_2,_1) = 0.0
1
paper_schedule_(0,_3,_0) = 0.0
1
paper_schedule_(0,_3,_1) = 0.0
1
paper_schedule_(0,_4,_0) = 0.0
1
paper_schedule_(0,_4,_1) = 0.0
1
paper_schedule_(0,_5,_0) = 0.0
1
paper_schedule_(0,_5,_1) = 0.0
1
paper_schedule_(0,_6,_0) = 0.0
1
paper_schedule_(0,_6,_1) = 0.0
1
paper_schedule_(0,_7,_0) = 0.0
1
paper_schedule_(0,_7,_1) = 0.0
1
paper_schedule_(0,_8,_0) = 0.0
1
paper_schedule_(0,_8,_1) = 0.0
1
paper_schedule_(0,_9,_0) = 0.0
1
paper_schedule_(0,_9,_1) = 0.0
1
paper_schedule_(1,_0,_0) = 0.0
1
paper_schedule_(1,_0,_1) = 0.0
1
paper_schedule_(1,_1,_0) = 0.0
1
paper_schedule_(1,_1,_1) = 0.0
1
pa